In [1]:
import os, torch
import shutil # shutil 모듈을 사용하여 파일 및 디렉토리 복사, 이동, 삭제 등을 수행할 수 있습니다.
DIR1 = './results/temp_1'
DIR2 = './results/temp_2'
DIR = '_'.join(DIR1.split('_')[:-1])
DIR = DIR.replace('temp', 'latency')

In [2]:
### 1. 새로운 디렉토리 생성
if not os.path.exists(DIR):
    os.makedirs(DIR)
else:
    # 디렉토리가 이미 존재하는 경우 예외 처리
    shutil.rmtree(DIR)
    print(f"Directory {DIR} already exists. Removing it and creating a new one.")
    os.makedirs(DIR)

### 2. checkpoint 및 log, eval_results 디렉토리 복사
os.makedirs(os.path.join(DIR, 'checkpoints'), exist_ok=True)
os.makedirs(os.path.join(DIR, 'logs'), exist_ok=True)
os.makedirs(os.path.join(DIR, 'eval_results'), exist_ok=True)

Directory ./results/latency already exists. Removing it and creating a new one.


In [3]:
### dir 보기 
"""
---DIR1
    ---checkpoints
    ---eval_results
    ---logs (folder+file)
"""
### 3. checkpoint 파일 이동
def copy_files(init_dir, to_dir):
    # checkpoints
    init_checkpoint_dir = os.path.join(init_dir, 'checkpoints')
    to_checkpoint_dir = os.path.join(to_dir, 'checkpoints')
    checkpoints = os.listdir(init_checkpoint_dir) # checkpoint dir의 파일 리스트
    for checkpoint in checkpoints:
        init_checkpoint_path = os.path.join(init_checkpoint_dir, checkpoint)
        to_checkpoint_path = os.path.join(to_checkpoint_dir, checkpoint)
        if os.path.isfile(init_checkpoint_path):
            shutil.copy2(init_checkpoint_path, to_checkpoint_path)
        else:
            raise FileNotFoundError(f"{init_checkpoint_path} is not a file.")
    # eval_results
    init_eval_results_dir = os.path.join(init_dir, 'eval_results')
    to_eval_results_dir = os.path.join(to_dir, 'eval_results')
    eval_results = os.listdir(init_eval_results_dir) # eval_results dir의 파일 리스트
    for eval_result in eval_results:
        init_eval_result_path = os.path.join(init_eval_results_dir, eval_result)
        to_eval_result_path = os.path.join(to_eval_results_dir, eval_result)
        if os.path.isfile(init_eval_result_path):
            shutil.copy2(init_eval_result_path, to_eval_result_path)
        else:
            raise FileNotFoundError(f"{init_eval_result_path} is not a file.")
    # logs
    init_logs_dir = os.path.join(init_dir, 'logs')
    to_logs_dir = os.path.join(to_dir, 'logs')
    logs = os.listdir(init_logs_dir) # logs dir의 파일 리스트 --> 폴더이다. 
    for log in logs:
        init_log_path = os.path.join(init_logs_dir, log)
        to_log_path = os.path.join(to_logs_dir, log)
        if os.path.isdir(init_log_path):
            shutil.copytree(init_log_path, to_log_path) # copytree는 디렉토리와 그 안의 모든 파일을 복사합니다.
        else:
            raise FileNotFoundError(f"{init_log_path} is not a directory.")
    return None

copy_files(DIR1, DIR)
copy_files(DIR2, DIR)
print("All files copied successfully.")


All files copied successfully.


### 모델 불러오기 

In [4]:
import torch.nn as nn
import model2, model

def get_model_classes():
    """
    model 폴더 내에서 nn.Module 기반 클래스만 자동으로 dict로 반환 (instance가 아니라 class 반환)
    """
    model_classes = {}
    for k in dir(model2):
        obj = getattr(model2, k)
        if isinstance(obj, type) and issubclass(obj, nn.Module) and obj.__module__.startswith('model2.'):
            model_classes[k] = obj  # <-- instance() 하지 않고 class 자체만 저장

    for k in dir(model):
        obj = getattr(model, k)
        if isinstance(obj, type) and issubclass(obj, nn.Module) and obj.__module__.startswith('model.'):
            model_classes[k] = obj # <-- instance() 하지 않고 class 자체만 저장
    return model_classes

model_classes = get_model_classes()
print("Model classes loaded successfully for len(model_classes):", len(model_classes))
print("Model classes:", model_classes.keys())
    

Model classes loaded successfully for len(model_classes): 5
Model classes: dict_keys(['SlimDeepCAE_Bottleneck16_Dropout', 'SlimDeepCAE_Bottleneck8_Dropout', 'GANomaly', 'SlimDeepCAE_Bottleneck32_Dropout', 'SlimDeepCAE_Combo'])


### DATA LOAD

In [5]:
# Fashion MNIST dataset
import torchvision.transforms as transforms
import torch 
from torchvision import datasets
trainset = datasets.FashionMNIST(
    root      = './.data/', train = True,
    download  = True,
    transform = transforms.ToTensor())
testset = datasets.FashionMNIST(
    root      = './.data/', train     = False,
    download  = True,
    transform = transforms.ToTensor())

In [6]:
SELECT_NORMAL = 2 # Set 2 class as train dataset.
trainset.data = trainset.data[trainset.targets == SELECT_NORMAL]
trainset.targets = trainset.targets[trainset.targets == SELECT_NORMAL] # Set 2 class as train dataset.

test_label = [2,4,6] # Define actual test class that we use
actual_testdata = torch.isin(testset.targets, torch.tensor(test_label))
testset.data = testset.data[actual_testdata]
testset.targets = testset.targets[actual_testdata]

test_loader = torch.utils.data.DataLoader(
    dataset     = testset, batch_size  = 1,
    shuffle     = False,num_workers = 2)

train_data_size = len(trainset)
test_data_size = len(testset)

print("Train data size:", train_data_size, "Test data size:", test_data_size)

Train data size: 6000 Test data size: 3000


In [7]:
# 데이터셋을 먼저 train과 val로 나누고, train에 대해서만 증강을 적용
n_val = int(len(trainset) * 0.2)
n_train = len(trainset) - n_val
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

_, valset = torch.utils.data.random_split(trainset, [n_train, n_val], generator=torch.Generator().manual_seed(2025))
# valset은 증강을 적용하지 않음
val_loader = torch.utils.data.DataLoader(
    dataset     = valset, batch_size = 1,
    shuffle     = False,num_workers = 0)

### EVAL ALL

In [8]:
import matplotlib.pyplot as plt
import numpy as np
class Visualizer:
    def __init__(self, test_dataset, test_loader, model, device):
        self.test_dataset = test_dataset
        self.test_loader = test_loader
        self.model = model
        self.device = device

    def pick_random_samples(self, num_samples=2):
        test_2 = testset.data[testset.targets == 2]
        test_4 = testset.data[testset.targets == 4]
        test_6 = testset.data[testset.targets == 6]
        data_test_2 = test_2.view(test_2.size(0), -1).float() / 255.0
        data_test_4 = test_4.view(test_4.size(0), -1).float() / 255.0
        data_test_6 = test_6.view(test_6.size(0), -1).float() / 255.0
        data_test_2 = data_test_2[:num_samples]
        data_test_4 = data_test_4[:num_samples]
        data_test_6 = data_test_6[:num_samples]
        return [data_test_2, data_test_4, data_test_6]
    
    def visualize(self, num_samples=2):
        data_samples = self.pick_random_samples(num_samples)
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))

        for ax, data_sample, label in zip(axes, data_samples, [2, 4, 6]):
            ax.imshow(data_sample[0].view(28, 28).cpu().numpy(), cmap='gray')
            ax.set_title(f"Class {label}")
            ax.axis('off')

        plt.tight_layout()
    def visualize_reconstruction(self, num_samples=2):
        # Reshape the input data to match the expected dimensions for the model
        data_samples = self.pick_random_samples(num_samples)
        data_samples = [data_sample.view(data_sample.size(0), 1, 28, 28) for data_sample in data_samples]
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))

        for ax, data_sample, label in zip(axes, data_samples, [2, 4, 6]):
            data_sample = data_sample.to(self.device)
            with torch.no_grad():
                if hasattr(self.model, 'T'):
                    t = torch.randint(0, self.model.T, (data_sample.size(0),), device=self.device)
                    output = self.model(data_sample, t) # for diffusion model
                else:
                    output = self.model(data_sample) # for other models
                    if isinstance(output, tuple):
                        output = output[0] # for diffusion model
            reconstructed = output.view(-1, 28, 28) # Reshape the output to match the image dimensions
            reconstructed = reconstructed.cpu()

            ax.imshow(reconstructed[0].view(28, 28).numpy(), cmap='gray')
            ax.set_title(f"Class {label} - Reconstructed")
            ax.axis('off')

        plt.tight_layout()
        

In [9]:
def get_model_instance(pth_path, model_map:dict, device=None):
    """
    model_name에 해당하는 모델 클래스의 인스턴스를 생성하여 반환
    """
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1. 파일 이름 추출 (without extension)
    filename = os.path.basename(pth_path)  # ex: SlimDeepCAE_Dropout_MSEandMS-SSIMfp16.pth
    base_name = filename[:-4]  # remove ".pth"

    # 2. 모델 이름 추출 (Loss/precision 제외한 앞부분)
    model_name = base_name.split("_")[:-1] 
    model_name = "_".join(model_name)  # e.g., SlimDeepCAE_Dropout
    # if "_" in base_name:
    #     model_name = "_".join(base_name.split("_")[:2])  # e.g., SlimDeepCAE_Dropout

    # 3. 모델 클래스 찾기
    if model_name not in model_map:
        raise ValueError(f"Unknown model name: {model_name} — 확인된 모델: {list(model_map.keys())}")

    model_class = model_map[model_name]

    # 4. 모델 인스턴스 생성 및 weight 로딩
    model = model_class().to(device)
    state_dict = torch.load(pth_path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()

    return model

In [10]:
from loss.losses import FlexibleLoss

reconstruction_loss = {
    "MSE": FlexibleLoss(mode="mse",reduction="none"),
    "MSE+Gradient": FlexibleLoss(mode="mse+gradient", beta=1.0, gamma=0.1, reduction="none"),
    "MSE+MS-SSIM": FlexibleLoss(mode="mse+ms-ssim", beta=1.0, alpha=0.3,reduction="none"),
    "Charbonnier+MS-SSIM": FlexibleLoss(mode="charbonnier+ms-ssim", beta=1.0, alpha=0.5, reduction="none"),
    "Charbonnier+Gradient": FlexibleLoss(mode="charbonnier+gradient", beta=1.0, gamma=0.1, reduction="none"),
}


In [11]:
from eval_module.grid_loss import GridLossEvaluator
import pandas as pd
class GridEvaluator:
    def __init__(self, val_loader, test_loader, checkpoint_dir, model_map, loss_fns=None, device=None):
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.checkpoint_dir = checkpoint_dir
        print("Checkpoint directory:", checkpoint_dir)
        self.model_map = model_map
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.visualizer =None
        self.loss_fns = loss_fns
        self.results = []


    def paths(self):
        path_list = os.listdir(self.checkpoint_dir) # checkpoint dir의 파일 리스트
        path_list = [os.path.join(self.checkpoint_dir, path) for path in path_list if path.endswith('.pth')]
        return path_list
    
    def run(self):
        path_list = self.paths()
        for path in path_list:
            model = get_model_instance(path, self.model_map, self.device)
            gridevaluator = GridLossEvaluator(
                val_loader=self.val_loader,
                test_loader=self.test_loader,
                model=model,
                loss_fns=self.loss_fns,
                device=self.device,
                percentile=1.00
            )
            df = gridevaluator.run()
            self.results.append(df)
            print(f"Evaluation completed for {model.__class__.__name__} with path {path}")

    def save_results(self, output_dir='./eval_results'):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        combined_df = pd.concat(self.results, ignore_index=True) # Concatenate all DataFrames into one
        combined_df.to_csv(os.path.join(output_dir, 'evaluation_results.csv'), index=False)


In [12]:
grid = GridEvaluator(
    val_loader=val_loader,
    test_loader=test_loader,
    checkpoint_dir=os.path.join(DIR, 'checkpoints'),
    model_map=model_classes,
    loss_fns=reconstruction_loss,
    device=DEVICE,
)
grid.run()

Checkpoint directory: ./results/latency\checkpoints
>> Using loss: MSE


Scoring (MSE): 100%|██████████| 1200/1200 [00:00<00:00, 1312.68it/s]


Threshold: 0.1065


Scoring (MSE): 100%|██████████| 3000/3000 [00:04<00:00, 646.65it/s] 


 ROC-AUC: 0.6249 | PR-AUC: 0.7429 | F1: 0.0020 | ACC: 0.3340
>> Using loss: MSE+Gradient


...

 ROC-AUC: 0.6304 | PR-AUC: 0.7444 | F1: 0.0010 | ACC: 0.3337
>> Using loss: Charbonnier+Gradient


Scoring (Charbonnier+Gradient): 100%|██████████| 1200/1200 [00:01<00:00, 989.90it/s]


Threshold: 0.2778


Scoring (Charbonnier+Gradient): 100%|██████████| 3000/3000 [00:06<00:00, 495.07it/s]

 ROC-AUC: 0.5957 | PR-AUC: 0.7202 | F1: 0.0010 | ACC: 0.3337
Evaluation completed for SlimDeepCAE_Combo with path ./results/latency\checkpoints\SlimDeepCAE_Combo_MSEfp16.pth


In [13]:
grid.save_results(output_dir=os.path.join(DIR, 'eval_results'))